# Constants

In [1]:
#
# Constants
#  
COLAB = False              # False = local run, True = Google Colab run

#
# check if we're running Google Colab or not
#
if (COLAB):

  import google
  google.colab.drive.mount('/content/drive', force_remount=True)
  
  ROOT_DIR = '/content/drive/MyDrive/google_colab/Comp Theory/TuringMachine'

else:
  ROOT_DIR = '/Users/pworthjr/Documents/FAU/Comp Theory/turing_machine'

CFG_DIR = ROOT_DIR + '/config/'
IMG_DIR = ROOT_DIR + '/img/'


MAX_OPS = 250
DEBUG = True

# TM Classes

The Turing Machine and Tape classes were originally sourced from https://python-course.eu/applications-python/turing-machine.php
although we added support for definitional output and file parsing of transitions among other capabilities

In [2]:
class Tape(object):
    
    #blank_symbol = " "
    
    def __init__(self,
                 tape_string = "",
                 blank_symbol = " ", debug=False):
        
        self.__tape = dict((enumerate(tape_string)))
        
        # last line is equivalent to the following three lines:
        #self.__tape = {}
        #for i in range(len(tape_string)):
        #    self.__tape[i] = input[i]
         
        self.__blank_symbol = blank_symbol
        
        if (debug):
            print('Tape::tape[' + str(self.__tape) + ']')
            print('Tape::blank_symbol[' + str(self.__blank_symbol) + ']')
        
        
    def get_blank_symbol(self):
        return self.__blank_symbol
    
    def __str__(self):
        s = ""
        min_used_index = min(self.__tape.keys()) 
        max_used_index = max(self.__tape.keys())
        for i in range(min_used_index, max_used_index+1):
            s += self.__tape[i]
        return s    
   
    def __getitem__(self,index):
        if index in self.__tape:
            return self.__tape[index]
        else:
            return self.__blank_symbol

    def __setitem__(self, pos, char):
        self.__tape[pos] = char 

In [3]:
#
# originally sourced from https://python-course.eu/applications-python/turing-machine.php
# with updates to support serializstion (output) and parsing (input)
#
import string

class TuringMachine(object):
    
  def __init__(self, 
                tape = "", 
                blank_symbol = " ",
                Sigma = " ",
                Gamma = " ",
                states = None,
                initial_state = "",
                final_states = None,
                transition_file = None,
                transition_function = None):
                
    self.__tape = Tape(
       tape_string=tape, 
       blank_symbol=blank_symbol, 
       debug=True)
    
    self.__head_position = 0
    self.__blank_symbol = blank_symbol
    
    self.__Sigma = Sigma
    self.__Gamma = Gamma
    
    self.__states = states
    self.__initial_state =  initial_state
    self.__current_state = initial_state
    
    if (transition_function == None):
        self.read_transition_rules(transition_file)
    else:
        self.__transition_function = transition_function
    
    if final_states == None:
        self.__final_states = set()
    else:
        self.__final_states = set(final_states)


  def get_tape(self): 
    return str(self.__tape)

  def get_tape_blank(self):
     return self.__tape.get_blank_symbol()

  def final(self):
    if self.__current_state in self.__final_states:
        #print('Accepting')
        return True
    else:
        #print('Not Acceping')
        return False
    

  def step(self, debug=False):

    char_under_head = self.__tape[self.__head_position]
    x = (self.__current_state, char_under_head)
    
    if x in self.__transition_function:
        y = self.__transition_function[x]

        if (debug):
          print('transition function: ' + str(x) + ' -> ' + str(y))
          print("tape before operation: " + self.get_tape() + '; [head_pos: ' + str(self.__head_position) + ']')
    
        self.__tape[self.__head_position] = y[1]
        if y[2] == "R":
            self.__head_position += 1
        elif y[2] == "L":
            self.__head_position -= 1
        self.__current_state = y[0]
    else:
       if (debug):
          print('*** WARNING: ' + str(x) + ' not in transition function set ***')
          return False

    if (debug):
      print("tape after operation: " + self.get_tape() + '; [head_pos: ' + str(self.__head_position) + ']')
      print('---------------------------------------------------------')
      
    return True
  
  
  #
  # computes the input_str with a max_ops value as input 
  #
  def compute(self, input_string=None, max_ops=MAX_OPS, debug=False):

    self.__tape = Tape(
       tape_string=input_string, 
       blank_symbol=self.__blank_symbol
       )
    
    if (debug):
       print('TuringMachine::compute(tape=' + self.get_tape() + ', blank_symbol=' + str(self.get_tape_blank()) + ')')
       print('---------------------------------------------------------')

    ops = 0
    while not self.final():
      valid_state = self.step(debug=debug)
      ops += 1

      if not valid_state:
        print('*** not a valid state, therefore not accepting ***')
        return False

      if (ops >= max_ops):
        if (debug):
          print('*** WARNING: not accepting after max_ops('+str(ops)+') operations ***')
        return False
    
    if (debug):
       print("ACCEPTING after " + str(ops) + ' operations.')
       print("TM Tape Output: " + self.get_tape())    

    return True


  #
  # support function for parsing transitions from file
  #
  def split_equation(self, text):
    parts = text.split("=")
    lhs = parts[0].strip()
    rhs = parts[1].strip()
    return lhs, rhs
  

  #
  # support function for parsing transitions from file
  #
  def parse_transition_rule(self, rule):
    
    # Remove whitespace and split into components
    rule = rule.strip().replace(" ", "").replace("δ", "").replace("{", "").replace("}", "")

    lhs, rhs = self.split_equation(rule)    

    lhs = lhs.replace('(', '').replace(')', '')
    rhs = rhs.replace('(', '').replace(')', '')

    # Extract state and symbol from strings
    q, sym = lhs.split(',')
    q_next, sym_next, direction = rhs.split(',')
    
    return (q, sym), (q_next, sym_next, direction)
  
  #
  # read transition rules from file
  # takes filename as input, returns rules
  # 
  def read_transition_rules(self, filename):
    rules = {}
    with open(filename, "r") as f:
        for line in f:
            if line.strip() != "":
                key, value = self.parse_transition_rule(line)
                rules[key] = value
        
        self.__transition_function = rules


  def print(self):
    print("--------------- ** TM Definition ** ---------------")
    print("Q = {" + ", ".join(self.__states) + "}")
    print("Σ = {" + ", ".join(self.__Sigma) + "}")
    print("Γ = {" + ", ".join(self.__Gamma) + "}")
    print("δ : ")
    for key, val in self.__transition_function.items():
        print('   δ'+str(key)+' = {'+str(val)+ str('}'))
    print("q0 = " + self.__initial_state)
    print("b = " + self.__blank_symbol)
    print("F = {" + ", ".join(self.__final_states) + "}")
    print("---------------------------------------------------")


# Examples

## Initialize TM in Code

In [4]:
states = {"init", "final"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "init",

accepting_states = ["final"],

transition_function = {("init","0"):("init", "1", "R"),
                       ("init","1"):("init", "0", "R"),
                       ("init","□"):("final","□", "N"),
                       }

final_states = {"final"}

t = TuringMachine(states = states,
                  Gamma = Gamma,
                  Sigma = Sigma,
                  blank_symbol="□",
                  initial_state = "init",
                  final_states = final_states,
                  transition_function=transition_function)

t.print()

if (t.compute("010011001□", debug=DEBUG)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')  
    

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {final, init}
Σ = {1, 0}
Γ = {1, 0, □}
δ : 
   δ('init', '0') = {('init', '1', 'R')}
   δ('init', '1') = {('init', '0', 'R')}
   δ('init', '□') = {('final', '□', 'N')}
q0 = init
b = □
F = {final}
---------------------------------------------------
TuringMachine::compute(tape=010011001□, blank_symbol=□)
---------------------------------------------------------
transition function: ('init', '0') -> ('init', '1', 'R')
tape before operation: 010011001□; [head_pos: 0]
tape after operation: 110011001□; [head_pos: 1]
---------------------------------------------------------
transition function: ('init', '1') -> ('init', '0', 'R')
tape before operation: 110011001□; [head_pos: 1]
tape after operation: 100011001□; [head_pos: 2]
---------------------------------------------------------
transition function: ('init', '0') -> ('init', '1', 'R')
tape before operation: 100011001□; [head_pos: 2]
tape after oper

## TM from File

### L = L(*a*\**b*\*)

In [5]:
#
# From Module 10 slides:
# For the alphabet (Sigma) = {a, b} design a Turing machine 
# that accepts the language denoted by the regular expression 'a*b*'
#
states = {"q0", "q1"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "q0",
accepting_states = ["q1"]

final_states = {"q1"}

t = TuringMachine(states=states,
                  Sigma=Sigma,
                  Gamma=Gamma,
                  initial_state="q0",
                  blank_symbol="□",
                  final_states=final_states,
                  transition_file=CFG_DIR + '/tm10.1_v2.tm')

t.print()

if (t.compute("abbbbbb□", debug=DEBUG)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')
    

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {q0, q1}
Σ = {1, 0}
Γ = {1, 0, □}
δ : 
   δ('q0', 'a') = {('q0', 'a', 'R')}
   δ('q0', 'b') = {('q0', 'b', 'R')}
   δ('q0', '□') = {('q1', '□', 'R')}
q0 = q0
b = □
F = {q1}
---------------------------------------------------
TuringMachine::compute(tape=abbbbbb□, blank_symbol=□)
---------------------------------------------------------
transition function: ('q0', 'a') -> ('q0', 'a', 'R')
tape before operation: abbbbbb□; [head_pos: 0]
tape after operation: abbbbbb□; [head_pos: 1]
---------------------------------------------------------
transition function: ('q0', 'b') -> ('q0', 'b', 'R')
tape before operation: abbbbbb□; [head_pos: 1]
tape after operation: abbbbbb□; [head_pos: 2]
---------------------------------------------------------
transition function: ('q0', 'b') -> ('q0', 'b', 'R')
tape before operation: abbbbbb□; [head_pos: 2]
tape after operation: abbbbbb□; [head_pos: 3]
----------------

### L = L(00\*)

In [6]:
#
# From Module 10 slides:
# For the alphabet (Sigma) = {0, 1} design a Turing machine 
# that accepts the language denoted by the regular expression '00*'
#
states = {"q0", "q1"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "q0",
accepting_states = ["q1"]

final_states = {"q1"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  initial_state = "q0",
                  final_states = final_states,
                  blank_symbol="□",
                  transition_file=CFG_DIR + '/tm10.2_v2.tm')

t.print()

if (t.compute("000000000000000□", debug=DEBUG)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')
    

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {q0, q1}
Σ = {1, 0}
Γ = {1, 0, □}
δ : 
   δ('q0', '0') = {('q0', '0', 'R')}
   δ('q0', '□') = {('q1', '□', 'L')}
q0 = q0
b = □
F = {q1}
---------------------------------------------------
TuringMachine::compute(tape=000000000000000□, blank_symbol=□)
---------------------------------------------------------
transition function: ('q0', '0') -> ('q0', '0', 'R')
tape before operation: 000000000000000□; [head_pos: 0]
tape after operation: 000000000000000□; [head_pos: 1]
---------------------------------------------------------
transition function: ('q0', '0') -> ('q0', '0', 'R')
tape before operation: 000000000000000□; [head_pos: 1]
tape after operation: 000000000000000□; [head_pos: 2]
---------------------------------------------------------
transition function: ('q0', '0') -> ('q0', '0', 'R')
tape before operation: 000000000000000□; [head_pos: 2]
tape after operation: 000000000000000□; [head_pos: 

## Assignment 7 (module 10 & 11)

### L = L(*aaba*\**b*)

In [7]:
#
# From Assignment 7a, L = L(aaba*b)
#
states = {"q0", "q1", "q2", "q3", "q4"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "□", "x", "y", "z", "w"}

initial_state = "q0",
accepting_states = ["q4"]

final_states = {"q4"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  initial_state = "q0",
                  final_states = final_states,
                  blank_symbol="□",
                  transition_function=None,
                  transition_file=CFG_DIR + '/assign7_1a_04122023.txt')

t.print()

if (t.compute("aabaaaaaaaaaaaaaaab□", debug=DEBUG)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {q0, q1, q3, q4, q2}
Σ = {a, b}
Γ = {□, b, w, x, a, y, z}
δ : 
   δ('q0', 'a') = {('q1', 'x', 'R')}
   δ('q1', 'a') = {('q2', 'y', 'R')}
   δ('q2', 'b') = {('q3', 'z', 'R')}
   δ('q3', 'a') = {('q3', 'w', 'R')}
   δ('q3', 'b') = {('q4', '□', 'R')}
q0 = q0
b = □
F = {q4}
---------------------------------------------------
TuringMachine::compute(tape=aabaaaaaaaaaaaaaaab□, blank_symbol=□)
---------------------------------------------------------
transition function: ('q0', 'a') -> ('q1', 'x', 'R')
tape before operation: aabaaaaaaaaaaaaaaab□; [head_pos: 0]
tape after operation: xabaaaaaaaaaaaaaaab□; [head_pos: 1]
---------------------------------------------------------
transition function: ('q1', 'a') -> ('q2', 'y', 'R')
tape before operation: xabaaaaaaaaaaaaaaab□; [head_pos: 1]
tape after operation: xybaaaaaaaaaaaaaaab□; [head_pos: 2]
---------------------------------------------------------
tran

### L = { *w*: |*w*| is a multiple of 4 }

In [8]:
states = {"q0", "q1", "q2", "q3", "q4", "q5"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "□"}

initial_state = "q0"
accepting_states = ["q5"]

final_states = {"q5"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  blank_symbol="□",
                  initial_state = initial_state,
                  final_states = final_states,
                  transition_file=CFG_DIR + '/assign7_1c_04122023.txt')

t.print()

if (t.compute("ababaaaabbba□", debug=DEBUG)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {q0, q1, q3, q4, q5, q2}
Σ = {a, b}
Γ = {a, □, b}
δ : 
   δ('q0', 'a') = {('q1', 'a', 'R')}
   δ('q0', 'b') = {('q1', 'b', 'R')}
   δ('q1', 'a') = {('q2', 'a', 'R')}
   δ('q1', 'b') = {('q2', 'b', 'R')}
   δ('q2', 'a') = {('q3', 'a', 'R')}
   δ('q2', 'b') = {('q3', 'b', 'R')}
   δ('q3', 'a') = {('q4', 'a', 'R')}
   δ('q3', 'b') = {('q4', 'b', 'R')}
   δ('q4', '□') = {('q5', '□', 'R')}
   δ('q4', 'a') = {('q1', 'a', 'R')}
   δ('q4', 'b') = {('q1', 'b', 'R')}
q0 = q0
b = □
F = {q5}
---------------------------------------------------
TuringMachine::compute(tape=ababaaaabbba□, blank_symbol=□)
---------------------------------------------------------
transition function: ('q0', 'a') -> ('q1', 'a', 'R')
tape before operation: ababaaaabbba□; [head_pos: 0]
tape after operation: ababaaaabbba□; [head_pos: 1]
---------------------------------------------------------
transition function: ('q1', 'b') -> ('q

### $$ L = \{ a^nb^ma^{n+m} : n >= 0, m >= 1 \}$$

In [14]:
#
# From Assignment 7, 1f L = {anbman+m: n >= 0, m >=1 }
# annswer from textbook pg 429 (Section 9.1 question 8f)
#
states = {"q0", "q1", "q2", "q3", "q4", "q5", "q6"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "y", "x", "□"}

initial_state = "q0"
accepting_states = ["q6"]

final_states = {"q6"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  blank_symbol="□",
                  initial_state = initial_state,
                  final_states = final_states,
                  transition_file=CFG_DIR + '/assign7_1f_04122023.txt')

t.print()

if (t.compute("abbbbaaaaa□", debug=DEBUG)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {q0, q6, q1, q3, q4, q5, q2}
Σ = {a, b}
Γ = {□, b, x, a, y}
δ : 
   δ('q0', 'b') = {('q1', 'b', 'L')}
   δ('q0', 'a') = {('q1', 'b', 'R')}
   δ('q1', 'a') = {('q1', 'b', 'R')}
   δ('q1', 'b') = {('q1', 'b', 'L')}
   δ('q1', '□') = {('q2', '□', 'R')}
   δ('q2', 'b') = {('q3', 'y', 'R')}
   δ('q2', 'x') = {('q5', 'x', 'R')}
   δ('q3', 'x') = {('q3', 'x', 'R')}
   δ('q3', 'b') = {('q3', 'b', 'R')}
   δ('q3', 'a') = {('q4', 'x', 'L')}
   δ('q4', 'b') = {('q4', 'b', 'L')}
   δ('q4', 'x') = {('q4', 'x', 'L')}
   δ('q4', 'y') = {('q2', 'y', 'R')}
   δ('q5', 'x') = {('q5', 'x', 'R')}
   δ('q5', '□') = {('q6', '□', 'L')}
q0 = q0
b = □
F = {q6}
---------------------------------------------------
TuringMachine::compute(tape=abbbbaaaaa□, blank_symbol=□)
---------------------------------------------------------
transition function: ('q0', 'a') -> ('q1', 'b', 'R')
tape before operation: abbbbaaaaa□; [head_po

### L = { *ww* : w ϵ {*a*, *b*}\+ }

In [15]:
#
# From Assignment 7, 2 L = {ww : w is in {a, b}+}
#
states = {"q0", "q1", "q2", "q3"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "□"}

initial_state = "q0"
accepting_states = ["q3"]

final_states = {"q3"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  blank_symbol="□",
                  initial_state = initial_state,
                  final_states = final_states,
                  transition_file=CFG_DIR + '/assign7_2_04162023.txt')

t.print()

if (t.compute("aabb□", debug=DEBUG)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {q0, q3, q2, q1}
Σ = {a, b}
Γ = {a, □, b}
δ : 
   δ('q0', 'a') = {('q1', 'a', 'R')}
   δ('q0', 'b') = {('q1', 'b', 'R')}
   δ('q1', 'a') = {('q2', 'a', 'R')}
   δ('q1', 'b') = {('q2', 'b', 'R')}
   δ('q2', 'a') = {('q2', 'a', 'R')}
   δ('q2', 'b') = {('q2', 'b', 'R')}
   δ('q2', '□') = {('q3', '□', 'R')}
q0 = q0
b = □
F = {q3}
---------------------------------------------------
TuringMachine::compute(tape=aabb□, blank_symbol=□)
---------------------------------------------------------
transition function: ('q0', 'a') -> ('q1', 'a', 'R')
tape before operation: aabb□; [head_pos: 0]
tape after operation: aabb□; [head_pos: 1]
---------------------------------------------------------
transition function: ('q1', 'a') -> ('q2', 'a', 'R')
tape before operation: aabb□; [head_pos: 1]
tape after operation: aabb□; [head_pos: 2]
---------------------------------------------------------
transition function: 

### *f*(*w*) = $w^R$, where *w* ϵ {0, 1}$^+$

In [ ]:
#
# From Assignment 7, 3: construct a Turing Machine to 
# compute the function f(w) = wR
#
states = {"q0", "q1"}
Sigma = {"0", "1"}
Gamma = {"0", "1", "□"}

initial_state = "q0"
accepting_states = ["q1"]

final_states = {"q1"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  blank_symbol="□",
                  initial_state = initial_state,
                  final_states = final_states,
                  transition_file=CFG_DIR + '/assign7_3_04162023.txt')

t.print()

if (t.compute("0101010101□", debug=DEBUG)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

Tape::tape[{}]
Tape::blank_symbol[□]
--------------- ** TM Definition ** ---------------
Q = {q0, q1}
Σ = {1, 0}
Γ = {1, 0, □}
δ : 
   δ('q0', '1') = {('q0', '0', 'R')}
   δ('q0', '0') = {('q0', '1', 'R')}
   δ('q0', '□') = {('q1', '□', 'R')}
q0 = q0
b = □
F = {q1}
---------------------------------------------------
TuringMachine::compute(tape=0101010101□, blank_symbol=□)
---------------------------------------------------------
transition function: ('q0', '0') -> ('q0', '1', 'R')
tape before operation: 0101010101□; [head_pos: 0]
tape after operation: 1101010101□; [head_pos: 1]
---------------------------------------------------------
transition function: ('q0', '1') -> ('q0', '0', 'R')
tape before operation: 1101010101□; [head_pos: 1]
tape after operation: 1001010101□; [head_pos: 2]
---------------------------------------------------------
transition function: ('q0', '0') -> ('q0', '1', 'R')
tape before operation: 1001010101□; [head_pos: 2]
tape after operation: 1011010101□; [head_pos